In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from dateutil.relativedelta import relativedelta

In [169]:
air = pd.read_csv('./data/air.csv', engine = 'python', encoding='utf-8', index_col=0)
pop = pd.read_csv('./data/clean.csv', engine = 'python', encoding='utf-8', index_col=0)
covid = pd.read_csv('./data/covid_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
demand = pd.read_csv('./data/demand_month.csv', engine = 'python', encoding='utf-8', index_col=0)
kwrds = pd.read_csv('./data/kwrds_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
kwrds2 = pd.read_csv('./data/kwrds_2_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
mvpop = pd.read_csv('./data/mvpop.csv', engine = 'python', encoding='utf-8', index_col=0)
people = pd.read_csv('./data/people_total_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
male = pd.read_csv('./data/people_male_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
female = pd.read_csv('./data/people_female_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
distance = pd.read_csv('./data/social_distance_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
station = pd.read_csv('./data/station_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)
weather = pd.read_csv('./data/weather_cleaned.csv', engine = 'python', encoding='utf-8', index_col=0)

In [319]:
data_list = [air, pop, covid, demand, kwrds, kwrds2, mvpop, people, male, female, distance, station, weather]

In [252]:
for data in data_list:
    print(data.shape[0])

1461
1461
1461
1461
1461
1461
1461
1461
1461
1461
1461
1461
1461


air: 지역별  
pop: 지역별  
covid: 지역별 (total, new 구분)  
demand: 월별로 되어 있어 처리 필요  
kwrds, kwrds2: 전체  
mvpop: 지역별  
people, male, female: 지역별  
distance: 전체, 날짜 처리 필요, 열 삭제 필요  
station: 지역별, 날짜 처리 필요, 컬럼명 변경 필요  
weather: 전체  

# Data Cleaning

## demand

In [172]:
demand.reset_index(inplace=True)

In [173]:
demand.month = demand['month'].astype(str)
demand.month = demand['month'].apply(lambda e: dt.datetime.strptime(e, "%Y%m"))

In [174]:
demand.set_index('month', drop=True, inplace=True)

In [245]:
df = pd.DataFrame(index = pd.date_range('2018-01-01', '2021-12-31'), columns={'GW', 'DDM', 'SD', 'JL', 'sum'})
df = df[['GW', 'DDM', 'SD', 'JL', 'sum']]

In [210]:
for date_df, data_df in df.iterrows():
    for date_demand, data_demand in demand.iterrows():
        if date_df.year == date_demand.year and date_df.month == date_demand.month:
            data_df = data_demand.values

In [221]:
def substitute(df1, col1, df2, col2):
    for i_df1, row_df1 in df1.iterrows():
        for i_df2, row_df2 in df2.iterrows():
            if i_df1.year == i_df2.year and i_df1.month == i_df2.month:
                val = row_df2[col2]
                df.loc[i_df1, col1] = val

In [246]:
substitute(df, 'GW', demand, 'GW')
substitute(df, 'DDM', demand, 'DDM')
substitute(df, 'SD', demand, 'SD')
substitute(df, 'JL', demand, 'JL')
substitute(df, 'sum', demand, 'sum')

In [247]:
demand = df

## distance

In [175]:
distance.drop('Unnamed: 5', axis=1, inplace=True)
distance.drop('Unnamed: 6', axis=1, inplace=True)
distance.drop('Unnamed: 7', axis=1, inplace=True)
distance.drop('Unnamed: 8', axis=1, inplace=True)
distance.drop('Unnamed: 9', axis=1, inplace=True)

In [176]:
distance.reset_index(inplace=True)

distance.date = distance['date'].astype(str)
distance.date = distance['date'].apply(lambda e: dt.datetime.strptime(e, "%Y%m%d"))

distance.set_index('date', drop=True, inplace=True)

## station

In [177]:
station.columns = ['JL', 'GW', 'SD', 'DDM', 'four_districts_total']
station = station[['GW', 'DDM', 'SD', 'JL', 'four_districts_total']]

station.reset_index(inplace=True)

station.date = station['date'].astype(str)
station.date = station['date'].apply(lambda e: dt.datetime.strptime(e, "%m/%d/%Y"))

station.set_index('date', drop=True, inplace=True)

/var/folders/b3/zcggl_2158g9z4k11vk_wckw0000gn/T/ipykernel_77404/1087275548.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station.date = station['date'].astype(str)
/var/folders/b3/zcggl_2158g9z4k11vk_wckw0000gn/T/ipykernel_77404/1087275548.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station.date = station['date'].apply(lambda e: dt.datetime.strptime(e, "%m/%d/%Y"))


## 나머지 index datatype 바꿔주기

In [320]:
for data in data_list:
    print(type(data.index))

<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.base.Index'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.base.Index'>


In [321]:
# demand, distance, station 제외하고 나머지 datatype 바꿔줘야 함.

In [341]:
# air, kwrds2 인덱스 컬럼명 'date'로 변환

air.index.name = 'date'
kwrds2.index.name = 'date'

In [322]:
excluded_list = [air, pop, covid, kwrds, kwrds2, mvpop, people, male, female, weather]

In [345]:
for data in excluded_list:
    data.reset_index(inplace=True)

    data.date = data['date'].astype(str)
    data.date = data['date'].apply(lambda e: dt.datetime.strptime(e, "%Y-%m-%d"))
    
    data.set_index('date', drop=True, inplace=True)

In [346]:
for data in data_list:
    print(type(data.index))

<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


# merge

In [359]:
df_merge = pd.DataFrame(index = pd.date_range('2018-01-01', '2021-12-31'), columns=[[
    'GW', 'GW', 'GW', 'GW', 'GW', 'GW', 'GW', 'GW', 'GW', 'GW', 
    'DDM', 'DDM', 'DDM', 'DDM', 'DDM', 'DDM', 'DDM', 'DDM', 'DDM', 'DDM', 
    'SD', 'SD', 'SD', 'SD', 'SD', 'SD', 'SD', 'SD', 'SD', 'SD', 
    'JL', 'JL', 'JL', 'JL', 'JL', 'JL', 'JL', 'JL', 'JL', 'JL', 
    'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total', 'total'
],[
    'air', 'real_pop', 'pop', 'pop_male', 'pop_female', 'mvpop', 'covid_total', 'covid_new', 'new_user', 'station', 
    'air', 'real_pop', 'pop', 'pop_male', 'pop_female', 'mvpop', 'covid_total', 'covid_new', 'new_user', 'station', 
    'air', 'real_pop', 'pop', 'pop_male', 'pop_female', 'mvpop', 'covid_total', 'covid_new', 'new_user', 'station', 
    'air', 'real_pop', 'pop', 'pop_male', 'pop_female', 'mvpop', 'covid_total', 'covid_new', 'new_user', 'station', 
    '다이어트', '운동', '헬시플레저', '환경', '그린', '대기오염', '미세먼지', 'temp_avg', 'temp_low', 'temp_high', 'rain(mm)', 'relative_humidity', 'distance_stage', 'distance_total', 'distance_value', 'distance_nominal' 
]])

In [347]:
def substitute2(df1, col1, df2, col2):
    for i_df1, row_df1 in df1.iterrows():
        for i_df2, row_df2 in df2.iterrows():
            if i_df1 == i_df2:
                val = row_df2[col2]
                df1.loc[i_df1, col1] = val

In [348]:
substitute2(df_merge, ('GW', 'air'), air, 'GW')

In [352]:
district_list = ['GW', 'DDM', 'SD', 'JL']

In [363]:
for district in district_list:
    substitute2(df_merge, (district, 'air'), air, district)
    substitute2(df_merge, (district, 'real_pop'), pop, district)
    substitute2(df_merge, (district, 'pop'), people, district)
    substitute2(df_merge, (district, 'pop_male'), male, district)
    substitute2(df_merge, (district, 'pop_female'), female, district)
    substitute2(df_merge, (district, 'mvpop'), mvpop, district)
    substitute2(df_merge, (district, 'new_user'), demand, district)
    substitute2(df_merge, (district, 'station'), station, district)

In [368]:
substitute2(df_merge, ('GW', 'covid_total'), covid, 'GW_total')
substitute2(df_merge, ('GW', 'covid_new'), covid, 'GW_new')

substitute2(df_merge, ('DDM', 'covid_total'), covid, 'DDM_total')
substitute2(df_merge, ('DDM', 'covid_new'), covid, 'DDM_new')

substitute2(df_merge, ('SD', 'covid_total'), covid, 'SD_total')
substitute2(df_merge, ('SD', 'covid_new'), covid, 'SD_new')

substitute2(df_merge, ('JL', 'covid_total'), covid, 'JL_total')
substitute2(df_merge, ('JL', 'covid_new'), covid, 'JL_new')

In [367]:
kwrds_col_list = ['다이어트', '운동', '헬시플레저']
kwrds2_col_list = ['환경', '그린', '대기오염', '미세먼지']
weather_col_list = ['temp_avg', 'temp_low', 'temp_high', 'rain(mm)', 'relative_humidity']
distance_col_list = ['distance_stage', 'distance_total', 'distance_value', 'distance_nominal']

In [373]:
for col in kwrds_col_list:
    substitute2(df_merge, ('total', col), kwrds, col)

for col in kwrds2_col_list:
    substitute2(df_merge, ('total', col), kwrds2, col)
    
for col in weather_col_list:
    substitute2(df_merge, ('total', col), weather, col)
    
substitute2(df_merge, ('total', 'distance_stage'), distance, 'stage')
substitute2(df_merge, ('total', 'distance_total'), distance, 'total_stage')
substitute2(df_merge, ('total', 'distance_value'), distance, 'value')
substitute2(df_merge, ('total', 'distance_nominal'), distance, 'stage_nominal')

In [374]:
df_merge.to_csv('./data_merged.csv')